This project shows how to use a multimodal Gemini 2.0 flash model to assist structured information extraction task in Humanatarian context.

Reference -
https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_2_0_flash.ipynb

**Install Google Gen AI SDK for Python**

In [31]:
%pip install --upgrade --quiet google-genai

**Authenticate your notebook environment (Colab only)**

In [32]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

**Import Libraries**

In [33]:
from IPython.display import HTML, Markdown, display
from google import genai
from google.genai.types import (
    FunctionDeclaration,
    GenerateContentConfig,
    GoogleSearch,
    Part,
    Retrieval,
    SafetySetting,
    Tool,
    ToolCodeExecution,
    VertexAISearch,
)

### Set Google Cloud project information and create client

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [34]:
import os

PROJECT_ID = "PROJECT_ID"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

In [35]:
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

In [36]:
MODEL_ID = "gemini-2.0-flash-exp"  # @param {type: "string"}

In [37]:
response = client.models.generate_content(
    model=MODEL_ID, contents="What's the largest planet in our solar system?"
)

display(Markdown(response.text))

The largest planet in our solar system is **Jupiter**.


## Set system instructions

[System instructions](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/system-instruction-introduction) allow you to steer the behavior of the model. By setting the system instruction, you are giving the model additional context to understand the task, provide more customized responses, and adhere to guidelines over the user interaction.

In [38]:
system_instruction = """
  You are a helpful data analyst in humanatarian field.
  Your mission is to extract structured fields event type(Flood, Earthquake, armed conflict, etc) location (Geographical location mentioned in the text), affected population (Number of people impacted), and resource needed (Foold, shelter, medical aid, etc) in JSON format.
"""

**Data Extraction from a gvien text with Gemini 2.0**

In [39]:
prompt = """
  Text: Myanmar’s military unleashed a series of airstrikes and drone attacks on Buddhist monasteries in eastern Bago Region earlier this week, killing at least one woman and forcing hundreds of villagers to flee, according to resistance sources.

The attacks began on Monday and continued into the following day, targeting two villages in Bago’s Nyaunglebin Township, the sources said.

Nyaungpingyi, a resistance-held village east of the town of Nyaunglebin, was attacked on both days, while Pyinyae West, a village two miles to the north, was hit on Tuesday, a member of the anti-junta People’s Defence Force (PDF) told Myanmar Now.

Both villages are located near Puzunmyaung, a village in the Sittaung River Valley under the control of regime forces.

The first attack took place at around noon on Monday, targeting the Aye Myat Aung Monastery in Nyaungpingyi. Several buildings in and around the monastery were destroyed by bombs dropped from planes and drones, as well as by machine-gun fire, the PDF source said.

The next day, the Kyaung Thit Monastery in Nyaungpingyi was also bombed by a junta aircraft, according to a statement from the Karen National Union (KNU), which claimed that four civilians were injured during the attack.

The only confirmed fatality was reported in Pyinyae West, where a bomb dropped by a drone at around 12:30pm on Tuesday killed a 69-year-old woman living next to the village monastery.

The PDF source noted that the attacks coincided with the local bean harvest. “The people here grow beans for their livelihood. That’s why the military did this now,” he said.

The aerial assaults came just days after PDF forces based along the Sittaung River launched an attack on a junta outpost in Pazunmyaung, which is just a mile west of Nyaungpingyi.

The PDF succeeded in capturing a number of weapons in that battle, which took place on January 23, but withdrew without occupying the outpost. Both sides suffered casualties, sources said.

The regime often target schools, hospitals, and monasteries in resistance-held areas, as they are sometimes used by anti-junta fighters as temporary bases. Last Sunday, an attack on a village school in Mandalay Region’s Myingyan Township left 12 resistance fighters and six civilians dead.

In a statement released on Thursday, the regime denied that it had carried out airstrikes in Nyaunglebin or in neighbouring Shwegyin Township to the east. It claimed that resistance forces were responsible for any casualties or damage to property in these areas.
Shwegyin Township, which is largely under the control of the KNU’s Brigade 3, was among the first places in Myanmar to experience airstrikes in the wake of the February 2021 coup.
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(
        system_instruction=system_instruction,
    ),
)

display(Markdown(response.text))

```json
[
  {
    "event_type": "Armed conflict",
    "location": "Nyaunglebin Township, Bago Region, Myanmar",
    "affected_population": "Hundreds of villagers",
        "resource_needed": "Shelter"
  },
    {
    "event_type": "Armed conflict",
    "location": "Nyaungpingyi, Bago Region, Myanmar",
    "affected_population": "4 civilians injured",
        "resource_needed": "Medical Aid"
  },
    {
    "event_type": "Armed conflict",
    "location": "Pyinyae West, Bago Region, Myanmar",
       "affected_population": "1 fatality",
    "resource_needed": "Medical Aid"
  }
]
```


## Control generated output

[Controlled generation](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/control-generated-output) allows you to define a response schema to specify the structure of a model's output, the field names, and the expected data type for each field.

The response schema is specified in the `response_schema` parameter in `config`, and the model output will strictly follow that schema.

You can provide the schemas as [Pydantic](https://docs.pydantic.dev/) models or a [JSON](https://www.json.org/json-en.html) string and the model will respond as JSON or an [Enum](https://docs.python.org/3/library/enum.html) depending on the value set in `response_mime_type`.


In [40]:
from pydantic import BaseModel


class humanitarian_data(BaseModel):
    event_type: str
    location: str
    affected_population: int
    fatality_count: int
    resource_needed: list[str]


response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(
        system_instruction=system_instruction,
        response_mime_type="application/json",
        response_schema=humanitarian_data,
    ),
)

print(response.text)

{
    "affected_population": 500,
    "event_type": "armed conflict",
    "fatality_count": 1,
    "location": "Nyaunglebin Township, Bago Region",
    "resource_needed": ["shelter", "medical aid"]
}


### Send video from YouTube URL

In [46]:
video = Part.from_uri(
    file_uri="https://www.youtube.com/watch?v=GBAyRuf8juk",
    mime_type="video/mp4",
)
video_prompt = "Extract data for humanatarian support"
response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        video, video_prompt
    ],
    config=GenerateContentConfig(
    system_instruction=system_instruction,
    response_mime_type="application/json",
    response_schema=humanitarian_data,
    ),
)

display(Markdown(response.text))

{
  "affected_population":  400,
  "event_type": "Armed conflict",
  "fatality_count":  0,
  "location": "Ayeyarwady Region, Pathein Township, Bomie Village",
  "resource_needed": ["Shelter", "Food", "Medical aid"]
}

**Structured Information Extraction on Burmese language news**

You can use burmese language news as input to extract the structured data

In [42]:
burmese_prompt = """
  Text: တောင်တရုတ်ပင်လယ်နှင့် ဘင်္ဂလားပင်လယ်အော်မှ မုန်တိုင်းမိုးကြောင့် ရေကြီးမြေပြိုမှု ဖြစ်ခဲ့သော မန္တလေးတိုင်း ရမည်း သင်းမြို့နယ်၌ ယခုအခါ ရေပြန်ကျပြီဖြစ်၍ ကယ်ဆယ်ရေးလုပ်ငန်းများ လုပ်ဆောင်ခဲ့ရာ အလောင်း ၄၀ကျော် တွေ့ ထားပြီး လူ၁၀၀ကျော် ပျောက်နေကြောင်း ဒေသခံများနှင့် လူမှုကူညီ ရေးအသင်းများထံက သိရသည်။
စက်တင်ဘာ ၁၁ရက် ၁၂ရက်က အဆက်မပြတ်ရွာသည့် မိုးကြောင့် နာမည်ကျော် မိုးထိမိုးမိ ရွှေလုပ်ကွက်များရှိရာ ရမည်းသင်း အရှေ့ခြမ်း တောင်ပေါ်ရွာများဖြစ်သည့် ငါးတဲစု၊ ဝါးဖြူတောင် အထက်/အောက်၊ သပြေပင်၊ အရှေ့ပင်၊ ချောင်းဖျား၊ ရွှေတောမိုင်းအပါအဝင် ကျေးရွာ ၈ ရွာခန့်ရေကြီးနစ်မြုပ်ခဲ့ကာ တောင်ကျချောင်းရေ တိုက်စားမှုကြောင့် မြေပြိုမှုများလည်း ဖြစ်ပေါ်ခဲ့ခြင်း ဖြစ်သည်။
ရမည်းသင်းမြို့ လူမှုကူညီရေးအသင်းတခုမှ တာဝန်ရှိသူတဦးက “လက်ရှိ Dead Body က သယ်ထားတာက ၃၀ လောက်ပဲ ရသေးတယ်၊ အချို့ အလောင်းတွေက သယ်လို့မရတော့ အနီးနားက ရွာတွေမှာပဲ သဂြိုလ်လိုက်ကြတာမျိုး တွေလည်း ရှိ တယ်၊ မြင်းနားဘက်မှာဆိုရင် ရွာခံတွေ မြေမြုပ်ပေးလိုက်တာကို ၁၅ လောင်းကျော်ထိရှိတယ်၊ ရွာတွေက တောင်ပေါ်ဖြစ်တဲ့ အတွက်ကြောင့် မိုးအရမ်းများ ရေကြီးပြီး မြေပြိုတော့ လူတွေ အထိ အခိုက်သေဆုံးမှု တော်တော်များများ ရှိပါတယ်၊ ကျနော်တို့ဆီမှာ ရွာတွေကနေ ပေးထားတဲ့ ပျောက်ဆုံးစာရင်းက ၁၀၀ ကျော်လောက်ရှိတယ်”ဟု ပြောသည်။
လူသေဆုံးမှုနှင့် ပျက်စီးဆုံးရှုံးမှု အများဆုံးရွာမှာ သပြေပင်ကျေးရွာဖြစ်ပြီး နေအိမ်များ ရေစီးဖြင့် မျောပါသွားသလို မြေအောက်၌ နစ်မြုပ်သွားသောနေအိမ်များလည်း ရှိကြောင်း၊ အိမ်ခြေ ၃၅၀ ကျော်တွင် အမြင့်ပိုင်းမှ အိမ်ခြေ ၄၀ ကျော် သာ ကျန်ရှိတော့သလို ဒေသခံ ၈၀ ကျော် ပျောက်ဆုံးခဲ့သည့်အနက် ရုပ်အလောင်း ၃၀ ခန့် တွေ့ရှိထားပြီး ကျန်သူများမှာ ပျောက်ဆုံးနေဆဲဖြစ်ကြောင်း၊ ချောင်းဖျားကျေးရွာတွင်လည်း ရေကြီးရေလျှံမှုများ ဖြစ်ပေါ်ခဲ့ပြီး ပျောက်ဆုံးနေသူများလည်း ရှိကာ ဆက်လက် ရှာဖွေနေကြဆဲဟု သိရသည်။
ရမည်းသင်းမြို့ခံ လူမှုကူညီရေး အသင်းတခုမှ တာဝန်ရှိသူတဦးက “သပြေပင်ကျေးရွာရယ် ချောင်းဖျားရယ်ကတော့ တော်တော်ဆိုးဆိုးရွားရွားပဲ၊ ထိခိုက်ပျက်စီးမှုအများဆုံးနဲ့ ရွာရဲ့ ၃ ပုံ ၂ ပုံလောက်ကို ပါသွားတာ၊ သူက မြေပြိုပြီးပါသွားတာ၊ ချောင်းဘေးနားမှာ ရွာလေးတွေ တည်ပြီးနေကြတာ၊ ရွှေတောကို သွားတဲ့ လမ်းပေါ်မှာပေါ့၊ ရမည်းသင်းအရှေ့ဘက် ရွှေလုပ် ကွက်တွေရှိတဲ့ မိုးထိမိုးမိဘက်ပေါ့၊ ပြီးတော့ ငါးတပ်စုတို့ ဝါးဖြူတောင်တို့ ဘက်မှာလည်း ထိခိုက်ပျက်စီးမှုတွေ ရှိတယ်၊ အခုလက်ရှိ လမ်းပန်းဆက်သွယ်ရေးကလည်း တော်တော်လေးခက်ခဲနေတယ်၊ အဲဒီဘက်ကို သွားဖို့က မြေပြိုထားတာဆိုတော့ သွားလို့လာလို့က မလွယ်သေးဘူး”ဟု ပြောသည်။
အဆိုပါရွာများမှာ ရမည်းသင်းမြို့နယ်အတွင်း ဖြတ်သန်းစီးဆင်းသွားသည့် ပေါင်းလောင်းမြစ်ဘေးကျေးရွာများ ဖြစ်ကာ စက်တင်ဘာ ၁၁ ရက်က စတင်ပြီး မိုးသည်းထန်စွာ ရွာသွန်းသဖြင့် ရေဝင်ရောက်မှုဖြစ်ခဲ့ကာ ၁၂ ရက်တွင် ရေ ပြန်ကျသွား ကြောင်း၊ လက်ရှိအချိန်အထိ ယင်းကျေးရွာများသို့ စစ်ကောင်စီ၏ ကယ်ဆယ်ရေး ရောက်ရှိလာခြင်းမရှိဘဲ အသက်ရှင် ကျန်ရစ်သူများမှာလည်း စားဝတ်နေရေး ၊ အမိုးအကာနှင့် ကျန်းမာရေး ဒုက္ခများကြုံတွေ့နေရကြောင်း သိရသည်။

ဒေသခံတဦးက “တချို့နေရာတွေတော့ ရေပြန်ကျနေပါပြီ၊ ချောင်းဘေးနဲ့ တောင်ကျရေ နီးတဲ့ရွာတွေ ပိုပြီး ဒုက္ခများကြ တာပါ၊ ပြီးတော့ ကျီးနီကန်မကြီးကနေ ရေလွှတ်တာရယ် မိုးများပြီး တောင်ကျရေတွေကြောင့် မြေပြိုတာမျိုးတွေထိ ဖြစ်ခဲ့ တာမျိုးတွေပေါ့၊ လူတွေ ထိခိုက်သေဆုံးတာက မြေပြိုတာကြောင့်ပေါ့၊ အချို့ရွာတွေဆိုရင် အိမ်တွေ ပိုင်ဆိုင်မှုတွေ အားလုံး မြေအောက်ရောက်သွားကြတယ်၊ အတော်စိတ်မကောင်းစရာ ဖြစ်ရပ်တွေပါပဲ၊ ဒီနေ့အထိတော့ ပြည်သူအချင်းချင်း ကူညီ တာကလွဲပြီး တာဝန်ရှိသူတွေ တယောက်မှ ရောက်မလာသေးဘူး”ဟု ပြောသည်။

စစ်ကောင်စီဝါဒဖြန့်တယ်လီဂရမ်ချန်နယ်များတွင်မူ ရမည်းသင်းမြို့နယ်အတွင်း မိုးသည်းထန်စွာရွာသွန်းမှုကြောင့် မိုးထိမိုးမိ ရွှေလုပ်ကွက်များတွင် မြေပြိုကျသဖြင့် အလုပ်သမား အများအပြား ပျောက်ဆုံးနေပြီး သပြေပင်ရွာ အပါအဝင်အများအပြား မြေအောက်သို့ ရောက်သွားကာ သေဆုံးသူ ရာချီနှင့်အတူ လူ ၇၀၀ ခန့် ပျောက်ဆုံးနေကြောင်း၊ ရွှေမြင်တင် တောင်ပေါ်မှ တန်ခိုးကြီးဘုရားတဆူဖြစ်သော ရွှေမြင်တင် ဘုရားလည်း ပြိုကျသွားပြီး ထိခိုက်သေဆုံးမှုများ သည်ဟု ရေးသားဖြန့်ဝေထားကြသည်။

“၇၀၀ ကျော်လောက် ပျောက်နေတယ်ဆိုတာမျိုးတော့ မဟုတ်ပါဘူး၊ ၁၀၀ ကျော်လောက် ပျောက်ဆုံးတယ်ဆိုတာက အမှန်ပါ၊ ပြည်သူတွေအနေနဲ့ အရမ်းကြီး မစိုးရိမ်ကြပါနဲ့ ကျနော်တို့လည်း ရှာဖွေ ကယ်ဆယ်ရေးတွေ လုပ်နေပါတယ်ဗျ၊ တချို့ရွာတွေမှာတော့ ၃၀ ကျော်ကနေ ၅၀ ကျော်လောက်ထိ ပျောက်ဆုံးနေတာမျိုးတွေ ရှိတယ်ဆိုတော့ အများဆုံး ၁၀၀ ကျော်တော့ ရှိနိုင်ပါတယ်”ဟု ရမည်းသင်းမြို့ခံ လူမှုကူညီရေးအသင်းတခုမှ တာဝန်ရှိသူတဦးက ပြောသည်။
သပြေပင်ကျေးရွာမှ ဒေသခံများမှာ ကုန်းအမြင့် ပိုင်း၌ တိမ်းရှောင်နိုင်ခဲ့ကြသလို ပျောက်ဆုံးနေသူ ၈၀ ခန့်သာရှိကြောင်း၊ ကျန်ရစ်သော ဒေသခံများအနေဖြင့် လမ်းပန်း ဆက်သွယ်ရေးလွယ်ကူစေရန်အတွက် ရှင်းလင်းရေးလုပ်ငန်းများ စတင် လုပ်ကိုင်လျက်ရှိပြီး ယနေ့ နံနက်ပိုင်းတွင် ကျေးရွာအချို့သို့ ကယ်ဆယ်ရေးများ စတင်ဝင်ရောက်နိုင်ပြီဖြစ်ကြောင်း သိရသည်။

“ရွာက အိမ်တွေကတော့ ရေကြီးတည်းက အကုန်မျောပါသွားတာ မှန်ပါတယ်၊ လူတွေကတော့ ကုန်းအမြင့်တွေပေါ်မှာ ရှောင် နိုင်ခဲ့ကြပါတယ်၊ လက်ရှိရေကတော့ ကျသွားပါပြီ၊ ရေစီးနဲ့ ပါသွားတဲ့သူတွေကတော့ အခုထိ ရှင်လားသေလား မသိရသေးပါဘူး ရှာနေကြတုန်းပါ၊ မသိဘဲ ကောလဟာလတွေ မဖြန့်ပါနဲ့ဗျ”ဟု သပြေပင်ရွာသို့ အကူအညီများ သွားရောက်ပေးခဲ့သူတဦးက ပြောသည်။

ရမည်းသင်းမြို့နယ်တွင် ရေကြီးနစ်မြုပ် မြေပြိုမှုကြောင့် သေဆုံးသူ ၁၇ ဦး၏ ရုပ်အလောင်းများ တွေ့ရှိထားကြောင်း စက်တင်ဘာ ၁၂ ရက်နေ့တွင် စစ်ကောင်စီလက်အောက်ခံ မီးသတ်ဦးစီးဌာနမှ အသိပေးထုတ်ပြန်ထားသည်။

ရမည်းသင်းမြို့ပေါ်ရှိ လူမှုကူညီရေးအသင်းများအနေဖြင့် ရေဘေးကယ်ဆယ်ရေးအတွက် ကြိုတင်ပြင်ဆင်မှုများ ပြုလုပ် ထားသော်လည်း ကယ်ဆယ်ရေးလုပ်ငန်း အတွေ့အကြုံနည်းပါးမှုကြောင့် အခက်အခဲများလည်း ဖြစ်ပေါ်ခဲ့ရကြောင်း၊ မြို့ခံများနှင့် လူမှုကူညီရေးအသင်းများ ပူးပေါင်းဆောင်ရွက်မှုများ ရှိသော်လည်း အသက်ကယ်အင်္ကျီ၊ဘောကွင်းစသည့် ကယ်ဆယ်ရေးသုံး အထောက်အကူ ပစ္စည်းများ လုံလောက်မှု မရှိသလို အသက်ကယ်လှေ ၂ စီးခန့်သာ ရှိကြောင်း၊ ယခုအခါ ရှာဖွေကယ်ဆယ်ရေးလုပ်ငန်းများကို မြို့နယ်မီးသတ်ဦးစီးဌာန၏ ဦးဆောင်လမ်းညွှန်မှုဖြင့် မြို့ခံပရဟိတအသင်းများက ပူး ပေါင်းဆောင်ရွက်နေကြောင်း သိရသည်။

ရမည်းသင်းမြို့နယ်တွင် မြို့ပေါ်ရပ်ကွက် ၅ ခု၊ ကျေးရွာအုပ်စုပေါင်း ၆၃ အုပ်စု၊ ကျေးရွာပေါင်း ၂၄၇ ရွာ၊ လူဦးရေ ၂၅၈၀၉၁ ဦး ထိရှိပြီး အရှေ့ဘက်တွင် ရှမ်းပြည်နယ်ရှိ ပင်လောင်းမြို့နယ်၊ ကလောမြို့နယ်၊ တောင်ဘက်တွင် တပ်ကုန်းမြို့နယ်၊ အနောက်ဘက်တွင် မကွေးတိုင်းရှိ နတ်မောက်၊ မြို့သစ်မြို့နယ်၊ မြောက်ဘက်တွင် ပျော်ဘွယ်မြို့နယ်များ တည်ရှိသည်။

မြန်မာနိုင်ငံတွင် မုန်တိုင်းဒဏ်ကြောင့် မြို့နယ် ၃၄ ခုတွင် ရေကြီးရေလျှံမှုများ ဖြစ်ပွားခဲ့ပြီး လူဦးရေ ၂၃၆,၆၄၉ ဦးထိ ရေဘေးဒဏ်ခံစားခဲ့ရကာ ကူညီကယ်ဆယ်ရေးစခန်း ၁၈၇ ခုထိ ဖွင့်လှစ်ပေးထားကြောင်း၊ ရေကြီးနစ်မြုပ်မှုကြောင့် သေဆုံးသူ ၃၃ ဦးထိရှိသည် စစ်ကောင်စီ သတင်းမှန်ပြန်ကြားရေးအဖွဲ့က ထုတ်ပြန်ထားသည်။
"""

In [43]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=burmese_prompt,
    config=GenerateContentConfig(
        system_instruction=system_instruction,
        response_mime_type="application/json",
        response_schema=humanitarian_data,
    ),
)


In [44]:
display(Markdown(response.text))

{
  "affected_population": 258091,
  "event_type": "Flood and Landslide",
  "fatality_count": 40,
  "location": "Yamethin Township, Mandalay Region",
  "resource_needed": [
    "Food",
     "Shelter",
    "Medical aid",
     "Rescue equipment"
  ]
}